## Experiment 2: Using NLI to obtain constraints between WDIK facts and raw answers, and solving them using Z3 to flip answers if required

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import experiment_setup

In [3]:
from belief.evaluation import load_facts
from belief.nli import load_nli_model, load_nli_tokenizer, run_nli
from belief.lmbb import Proposition, LMBB
from tqdm import tqdm
import json
import random
from z3 import *

In [4]:
facts = load_facts('data/calibration_facts.json', num_batches=1)[0]

with open('cache/wdik.json', 'r') as f:
    wdik = json.load(f)

with open('data/constraints_v2.json', 'r') as f:
    constraint_data = json.load(f)
    
with open('cache/raw_outs_calib.json', 'r') as f:
    raw_outs = json.load(f)['outs']

In [5]:
evaluator = LMBB(
    model=None, 
    tokenizer=None, 
    raw_constraints=constraint_data['links'],
)

In [6]:
raw_outs_dict = {}
for out in raw_outs:
    raw_outs_dict[out['prop']] = {'yes': out['yes'], 'no': out['no']}

In [7]:
nli_model = load_nli_model()
nli_tokenizer = load_nli_tokenizer()

/home2/abhijit.manatkar/miniconda3/envs/advnlp/lib/python3.8/site-packages/transformers/convert_slow_tokenizer.py:454: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [25]:
z3mult = 1000
wdik_weight = 1.5
NUM_FACTS = 3

In [29]:
new_beliefs = {}

for prop in tqdm(facts):
    sent = prop.sentence
    raw_out = raw_outs_dict[sent]
    subject = prop.subject
    if NUM_FACTS > 0:
        wdik_facts = random.sample(wdik[subject], NUM_FACTS)
    else:
        wdik_facts = wdik[subject]
    
    bools = {}
    optim = Optimize()

    bools[sent] = Bool(sent)
    optim.add_soft(bools[sent], int(raw_out['yes'] * z3mult))
    optim.add_soft(Not(bools[sent]), int(raw_out['no'] * z3mult))

    assertion = prop.get_assertion()
    
    for wik in wdik_facts:
        bools[wik] = Bool(wik)
        optim.add_soft(bools[wik], wdik_weight * z3mult)

        # wik -> fact
        nli_out = run_nli(premise=wik, hypothesis=assertion, model=nli_model, tokenizer=nli_tokenizer)
        optim.add_soft(Implies(bools[wik], bools[sent]), int(nli_out['entailment'] * z3mult))
        optim.add_soft(Not(Implies(bools[wik], bools[sent])), int(nli_out['contradiction'] * z3mult))

        # fact -> wik
        nli_out = run_nli(premise=assertion, hypothesis=wik, model=nli_model, tokenizer=nli_tokenizer)
        optim.add_soft(Implies(bools[sent], bools[wik]), int(nli_out['entailment'] * z3mult))
        optim.add_soft(Not(Implies(bools[sent], bools[wik])), int(nli_out['contradiction'] * z3mult))

    optim.check()
    mod = optim.model()
    
    new_beliefs[sent] = Proposition.from_sent(sent, boolean=bool(mod.evaluate(bools[sent])))

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1072/1072 [02:09<00:00,  8.27it/s]


In [30]:
evaluator.set_beliefs(new_beliefs)
print("F1:", evaluator.calculate_f1(facts))
print("Consistency:", evaluator.calculate_consistency())

F1: 0.8305084695900029
Consistency: 0.971276159212146
